# Introduction <a name="introduction"></a>

The notebook will **create an empty external model for demonstration purposes**, **setup custom facts definitions** for it and show assets and model usecases utilities related to  **Custom facts** <br/>
For more information around helper utilities see [here](https://ibm-aigov-facts-client.mybluemix.net/)

It requires following services in the platform:

- Watson Knowledge catalog
- AI factsheet
- Facts client python sdk (>=1.0.42)

Sections: 

- [Setup](#setup)
    - [Cloud](#setupcloud)
    - [Cloud Pak for data](#setupcpd)
- [Initialize client](#init)
    - [Cloud](#setupcloudinit)
    - [Cloud Pak for data](#setupcpdinit)
- [Track custom model development facts](#trackdevfact)
- [Create external model properties](#createprop)
- [Save external model to catalog](#save)
- [Track model and model use case custom facts](#customfacts)
    - [Create custom facts definitions](#customfactsdef)
    - [Get custom facts definitions for model](#getcustomfactsdef)
        - [Set custom facts](#set)
        - [Get custom facts](#get)
        - [Get all facts](#get_all)
        - [Remove custom facts](#remove)
        - [Add to model usecase ](#add_mu)
    - [Create custom facts definitions for model usecase](#customfactsdefmu)
    - [Get custom facts definitions for model usecase](#getcustomfactsdefmu)
    - [Get model usecase ](#get_mu)
        - [Set custom facts](#set_mu_custom)
        - [Get custom facts](#get_mu_custom)
        - [Get all facts](#get_all_mu)
        - [Remove custom facts](#remove_mu_custom)
        - [Get tracked model usecases ](#get_tracked_mu)
- [Remove model from model usecase ](#remove_mu)
- [Cleanup](#clean)


<a name="setupcloud"></a>

# Setup<a name="setup"></a>
This notebook uses external models to illustrate custom facts. For it to work an administrator needs to enable support for external models on the `Manage` tab of the `Model Inventory` page. This will also make sure the platform asset catalog (PAC) is created.

### Imports

In [38]:
!pip install wget
!pip install pandas
!pip install -U ibm-aigov-facts-client


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Found existing installation: ibm-aigov-facts-client 1.0.42
Uninstalling ibm-aigov-facts-client-1.0.42:
  Successfully uninstalled ibm-aigov-facts-client-1.0.42
  Using cached ibm_aigov_facts_client-1.0.42-py3-none-any.whl (139 kB)

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [39]:
import warnings
import json
import wget
import pandas as pd
import shutil
import time
from IPython.core.display import display, Markdown
from ibm_aigov_facts_client import AIGovFactsClient
from ibm_aigov_facts_client.supporting_classes.factsheet_utils import DeploymentDetails,TrainingDataReference,ExternalModelSchemas

warnings.filterwarnings('ignore')
shutil.rmtree('./mlruns', ignore_errors=True)

- This sample will use IBM Cloud by default. If you prefer to use Cloud pak for data, set `use_cp4d=True`
- Flag `run_cleanup_at_end` offers option to delete created assets at the end of the notebook.The notebook will show URL to UI for model and model use case at certain cells. By dafault we set it to `run_cleanup_at_end=False` so you can access UI and see the changes. If you decide to cleanup assets at the end, set `run_cleanup_at_end=True` and remember cells showing links to UI will `NOT` work in that case.

In [40]:
use_cp4d=False
run_cleanup_at_end=False

[back to top](#introduction)


## IBM Cloud<a name="setupcloud"></a>

Your Cloud API key can be generated by going to the Users section of the Cloud console. From that page, go to **Manage->Access(IAM)->API keys-> Create**. Give your key a name and click Create, then copy the created key and use as API_KEY.

NOTE: You can also get OpenScale API_KEY using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: instruction

How to get api key using console:

```
bx login --sso
bx iam api-key-create 'my_key'
```

User apikey used below should have `EDITOR` access to platform asset catalog.

In [41]:
API_KEY="***"

<a name="setupcpd"></a>
## Cloud Pak for Data


- Service url is the Cloud pak for data platform host URL. For skytap environment, it would be the internal nginx URL.
- You can either use user password or platform apikey to authenticate

In [42]:
if use_cp4d:
    from ibm_aigov_facts_client import AIGovFactsClient,CloudPakforDataConfig

    creds=CloudPakforDataConfig(service_url="***",
        username="***",
        api_key="***")


In [43]:
model_identifier="PytorchExternalModelId"
model_name="PytorchExternalModel"
experiment_name="PytorchExperiment"

[back to top](#introduction)
# Client Initialization
<a name="init"></a>
## IBM Cloud<a name="setupcloudinit"></a>

- If running this notebook multiple times with same experiment name or anytime face error saying `Experiment with same name already exists`, use `set_as_current_experiment=True` when initiating client

In [44]:
if not use_cp4d:
    facts_client = AIGovFactsClient(api_key=API_KEY,experiment_name=experiment_name,external_model=True,enable_autolog=False)

2022/12/12 17:18:47 INFO : Experiment successfully created with ID 1 and name PytorchExperiment


## Cloud Pak for Data<a name="setupcpdinit"></a>

- If running this notebook multiple times with same experiment name or anytime face error saying `Experiment with same name already exists`, use `set_as_current_experiment=True` when initiating client

In [45]:
if use_cp4d:
    facts_client = AIGovFactsClient(experiment_name=experiment_name,external_model=True,cloud_pak_for_data_configs=creds,enable_autolog=False)

## Track development facts manually <a name="trackdevfact"></a>

You can collect different model training facts (i.e., training metrics, params and tags)using our facts client. For external models, facts can be captured using autolog or you can capture these facts manually if model is built with framework that is not supported for autolog. In that case, only option is to capture different model training facts and publish to factsheet manually. Steps to follow:

- Make sure to initiate client with `enable_autolog=False`
- Start tracing, ideally on top of the noetbook. It will intiate a run under provided experiment name
- Calculate and log metrics,params and tags as applicable
- get current run id
- Export the corresponding run payload to factsheet
- End tracing.

In this example, assume the model is built using regular `Pytorch` which is note supported for autolog so we will log some training facts manually and export to factsheet before saving the model stub.
For additonal information on manual logging, you can check [here](https://ibm-aigov-facts-client.mybluemix.net/#factsheet-manual-log-elements)

In [46]:
#start tracing 
facts_client.manual_log.start_trace()

#log matrics
facts_client.manual_log.log_metrics({"ROC training": 0.8, "F1 training": 0.9})

# log training params
p={"learning_rate": 0.01, "epochs": 10}
facts_client.manual_log.log_params(params=p)

# log tags related to model
facts_client.manual_log.set_tags({"engineering": "SaaS Platform",
        "release.version": "1.1.0"})

time.sleep(5)
# get current run id
current_run=facts_client.runs.get_current_run_id()

#export to factsheet
facts_client.export_facts.export_payload_manual(current_run)
#end tracing
facts_client.manual_log.end_trace()

2022/12/12 17:18:47 INFO : Manual tracing initiated successfully under run 5592724360b341e09d48cf57df639ae6
2022/12/12 17:18:47 INFO : logged metrics ['ROC training', 'F1 training'] successfully under run 5592724360b341e09d48cf57df639ae6
2022/12/12 17:18:47 INFO : logged params ['learning_rate', 'epochs'] successfully under run 5592724360b341e09d48cf57df639ae6
2022/12/12 17:18:47 INFO : logged tags ['engineering', 'release.version'] successfully under run 5592724360b341e09d48cf57df639ae6
2022/12/12 17:18:52 INFO : logging results to factsheet for run_id 5592724360b341e09d48cf57df639ae6
2022/12/12 17:18:53 INFO : Successfully logged results to Factsheet service for run_id 5592724360b341e09d48cf57df639ae6 under asset_id: 705b35a0-956c-4bf1-bd97-5f4a98ca69ad 
2022/12/12 17:18:53 INFO : Manual tracing ended successfully for run 5592724360b341e09d48cf57df639ae6


[back to top](#introduction)
## Create model properties<a name="createprop"></a>

To demonstrate custom facts we need to create a placeholder model on which we can show the APIs. 
The model itself does not matter for custom fact demonstration so we will just create a sample one quickly.
In a real end-to-end the model would be created based on real Sagemaker/Azure/Custom information and use real names and ids from these environments.

- Get all necessary details for the external model:
    - Input and Output schema details
    - Training data reference informations
    - Model deployment details from the platform where model is stored
- You can learn about these supporting classes in [here](https://ibm-aigov-facts-client.mybluemix.net/#helper-classes)

In [47]:
!rm external_model_facts_payload.json
wget.download("https://raw.githubusercontent.com/IBM/ai-governance-factsheet-samples/main/Assets/data/external_model_facts_payload.json")

'external_model_facts_payload.json'

In [48]:
with open("external_model_facts_payload.json", 'r') as f:
    payload = json.load(f)

In [49]:
external_schemas=ExternalModelSchemas(input=payload["schemas"]["input"],output=payload["schemas"]["output"])


In [50]:
# Derive training data reference schema from model training input schema

training_data_schema={}

input_schema= payload["schemas"]["input"][0]
training_data_schema["schema"]=input_schema
train_data_ref=TrainingDataReference(schema=training_data_schema)


You can get deployment details for model asset from platforms where model is built (i.e., AzureML, Sagemaker etc.) and save as metadata in model use case.
- ID: Deployment identifier specific to providers
- NAME: Name of the deployment
- TYPE: Deployment type
- SCORING ENDPOINT: Deployment scoring endpoint url

In [51]:
deployment_details=DeploymentDetails(identifier="098f6bcd4621d373cade4e832627b4f6"
                                    ,name="german_credit_risk_deployment"
                                    ,deployment_type="online"
                                    ,scoring_endpoint="test.iscore/score")

[back to top](#introduction)
# Save external model to catalog<a name="save"></a>

By default external model stubs are saved in the platform asset catalog which an administrator will have to enable.
See [the documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/factsheet-external.html) for more details.

In [52]:
external_model=facts_client.external_model_facts.save_external_model_asset(model_identifier=model_identifier,name=model_name
                        ,deployment_details=deployment_details
                        ,schemas=external_schemas
                        ,training_data_reference=train_data_ref
                        )

2022/12/12 17:18:58 INFO : External model asset saved successfully under asset_id c560f9d4-5ec7-431c-914b-394241f3f39a and catalog 745c21d7-ab6f-4408-9b3b-5d4760ccee1a
2022/12/12 17:18:58 INFO : Current model information: {'asset_id': 'c560f9d4-5ec7-431c-914b-394241f3f39a', 'container_type': 'catalog', 'container_id': '745c21d7-ab6f-4408-9b3b-5d4760ccee1a', 'facts_type': 'modelfacts_user'}


In [53]:
external_model.get_info(verbose=True)

{'name': 'PytorchExternalModel',
 'asset_type': 'model_stub',
 'url': 'https://dataplatform.cloud.ibm.com/data/catalogs/745c21d7-ab6f-4408-9b3b-5d4760ccee1a/asset/c560f9d4-5ec7-431c-914b-394241f3f39a/asset-preview?context=cpdaas',
 'asset_id': 'c560f9d4-5ec7-431c-914b-394241f3f39a',
 'container_type': 'catalog',
 'container_id': '745c21d7-ab6f-4408-9b3b-5d4760ccee1a',
 'facts_type': 'modelfacts_user'}

In [54]:
ui_url = external_model.get_info(verbose=True)["url"]
display(Markdown("[Click here to see the external model and logged development facts with it in UI](" + ui_url + ")"))

[Click here to see the external model and logged development facts with it in UI](https://dataplatform.cloud.ibm.com/data/catalogs/745c21d7-ab6f-4408-9b3b-5d4760ccee1a/asset/c560f9d4-5ec7-431c-914b-394241f3f39a/asset-preview?context=cpdaas)

[back to top](#introduction)
## Log model and model use case custom facts<a name="customfacts"></a> 


### Create custom facts definitions<a name="customfactsdef"></a> 

- User should have manage catalog permission
- Before you can set/get a custom fact you need to define the "slots" or "schema" of what custom facts you want to add to a model or model use case. In the defition you can specify the name, id and type for each fact "slot".
- Technically custom facts definitions are global asset attributes you want to see visible in your account. By default you create definitions under `modelfacts_user` that is associated to the model asset. You can also use `model_entry_user` as facts type to create definitions under model usecase.
- It can be of string, integer and date type only
- You can set list of strings if the attributes definition property `is_array` is set to True only.
- You can check the documentation to understand different options [here](https://ibm-aigov-facts-client.mybluemix.net/#factsheet-asset-elements)

In [55]:
!rm Asset_type_definition.csv
wget.download("https://raw.githubusercontent.com/IBM/ai-governance-factsheet-samples/main/Assets/data/Asset_type_definition.csv")

'Asset_type_definition.csv'

In [56]:
pd.read_csv('Asset_type_definition.csv')

,name,type,description,placeholder,is_array,required,hidden,readonly,default_value,label,minimum,maximum,min_length,max_length,is_searchable
0,TrainingData_Size,integer,Number of records in the training data,desc of userattrstr,False,True,False,False,0,TrainingData_Size,NaN,NaN,NaN,NaN,True
1,TrainingData_Ratio,string,Percentage of records in the training data,Percentage of records in the training data,False,True,False,False,0,TrainingData_Ratio,NaN,NaN,NaN,NaN,True
2,TestData_Size,integer,desc of userattrint,desc of userattrint,False,True,False,False,0,TestData_Size,NaN,NaN,NaN,NaN,False
3,TestData_Ratio,string,Percentage of records in the test data,Percentage of records in the test data,False,True,False,False,0,TestData_Ratio,NaN,NaN,NaN,NaN,True
4,Train_Class_Distributions,string,class distributions,class distributions,True,True,False,False,0,Train_Class_Distributions,NaN,NaN,NaN,NaN,False


In [57]:
facts_client.assets.create_custom_facts_definitions("Asset_type_definition.csv",overwrite=True)
time.sleep(5)

2022/12/12 17:19:00 INFO : Creating definitions for type modelfacts_user
2022/12/12 17:19:02 INFO : Custom facts definitions updated Successfully


### Get custom facts definitions<a name="getcustomfactsdef"></a> 

This will define new custom fact slots for sample custom facts like TrainingData_Size, TrainingData_Ratio, TrainingData_Size, TestData_Size, Train_Class_Distributions
You could define any kind of custom fact. But note that they are "global" i.e. they will be available as slots on every model in the same way.

In [58]:
facts_client.assets.get_facts_definitions(type_name="modelfacts_user")

{'description': 'The model fact user asset type to capture user defined attributes.',
 'fields': [{'key': 'TrainingData_Size',
   'type': 'integer',
   'facet': False,
   'is_array': False,
   'search_path': 'TrainingData_Size',
   'is_searchable_across_types': False},
  {'key': 'TrainingData_Ratio',
   'type': 'string',
   'facet': False,
   'is_array': False,
   'search_path': 'TrainingData_Ratio',
   'is_searchable_across_types': False},
  {'key': 'TestData_Ratio',
   'type': 'string',
   'facet': False,
   'is_array': False,
   'search_path': 'TestData_Ratio',
   'is_searchable_across_types': False}],
 'relationships': [],
 'properties': {'TrainingData_Size': {'type': 'integer',
   'description': 'Number of records in the training data',
   'placeholder': 'desc of userattrstr',
   'is_array': False,
   'required': True,
   'hidden': False,
   'readonly': False,
   'default_value': 0.0,
   'minimum': '',
   'maximum': '',
   'label': {'default': 'TrainingData_Size', 'en': 'TrainingD

[back to top](#introduction)

<a name="getmodel"></a> 
## Set custom facts<a name="set"></a> 
This will set the values for 4 of the custom fact "slots" define above

In [59]:
time.sleep(10)
external_model.set_custom_fact(fact_id="TrainingData_Size",value=5000)
external_model.set_custom_fact(fact_id="TrainingData_Ratio",value="70% of total")
external_model.set_custom_fact(fact_id="TestData_Size",value=1000)
external_model.set_custom_fact(fact_id="TestData_Ratio",value="30% of total")

2022/12/12 17:19:18 INFO : Custom fact TrainingData_Size successfully set to new value 5000
2022/12/12 17:19:19 INFO : Custom fact TrainingData_Ratio successfully set to new value 70% of total
2022/12/12 17:19:19 INFO : Custom fact TestData_Size successfully set to new value 1000
2022/12/12 17:19:19 INFO : Custom fact TestData_Ratio successfully set to new value 30% of total


`Train_Class_Distributions` attribute have `is_array` set to `True` thus allows to set string or list of strings.

In [60]:
external_model.set_custom_fact(fact_id="Train_Class_Distributions",value=["Risk 30%","No Risk 70%"])

2022/12/12 17:19:20 INFO : Custom fact Train_Class_Distributions successfully set to new value ['Risk 30%', 'No Risk 70%']


- You can set/update multiple facts at once and current values will be replaced

In [61]:
external_model.set_custom_facts({"TestData_Size": 1500, "TrainingData_Size": 5000, "Train_Class_Distributions":["Risk 35%","No Risk 65%"]})

2022/12/12 17:19:21 INFO : Custom facts ['TestData_Size', 'TrainingData_Size', 'Train_Class_Distributions'] successfully set to values [1500, 5000, ['Risk 35%', 'No Risk 65%']]


[back to top](#introduction)
## Get custom facts<a name="get"></a> 
- You can get all facts or fact by given name used in facts definitions. You can also visit the UI to view the customs facts set. You can get URL by running `external_model.get_info(verbose=True)["url"]`

In [62]:
ui_url = external_model.get_info(verbose=True)["url"]
display(Markdown("[Click here to see the external model and it's custom fact values in the UI](" + ui_url + ")"))

[Click here to see the external model and it's custom fact values in the UI](https://dataplatform.cloud.ibm.com/data/catalogs/745c21d7-ab6f-4408-9b3b-5d4760ccee1a/asset/c560f9d4-5ec7-431c-914b-394241f3f39a/asset-preview?context=cpdaas)

In [63]:
external_model.get_custom_fact_by_id(fact_id="TrainingData_Size")

5000.0

In [64]:
external_model.get_custom_facts()

{'TrainingData_Size': 5000.0,
 'TrainingData_Ratio': '70% of total',
 'TestData_Size': 1500.0,
 'TestData_Ratio': '30% of total',
 'Train_Class_Distributions': ['Risk 35%', 'No Risk 65%']}

[back to top](#introduction)
## Get all facts<a name="get_all"></a> 
- You can get all facts of all types (`modelfacts_user` or `model_entry_user`) or filter by any fact type.

In [65]:
external_model.get_facts_by_type()

{'TrainingData_Size': 5000.0,
 'TrainingData_Ratio': '70% of total',
 'TestData_Size': 1500.0,
 'TestData_Ratio': '30% of total',
 'Train_Class_Distributions': ['Risk 35%', 'No Risk 65%']}

In [66]:
external_model.get_all_facts()

{'metadata': {'usage': {'last_updated_at': '2022-12-12T22:19:21Z',
   'last_updater_id': 'IBMid-310002F0G1',
   'last_update_time': 1670883561045,
   'last_accessed_at': '2022-12-12T22:19:21Z',
   'last_access_time': 1670883561045,
   'last_accessor_id': 'IBMid-310002F0G1',
   'access_count': 0},
  'rov': {'mode': 0,
   'collaborator_ids': {},
   'member_roles': {'IBMid-310002F0G1': {'user_iam_id': 'IBMid-310002F0G1',
     'roles': ['OWNER']}}},
  'name': 'PytorchExternalModel',
  'tags': ['development'],
  'asset_type': 'model_stub',
  'origin_country': 'us',
  'rating': 0.0,
  'total_ratings': 0,
  'catalog_id': '745c21d7-ab6f-4408-9b3b-5d4760ccee1a',
  'created': 1670883399779,
  'created_at': '2022-12-12T22:16:39Z',
  'owner_id': 'IBMid-310002F0G1',
  'size': 0,
  'version': 2.0,
  'asset_state': 'available',
  'asset_attributes': ['model_stub',
   'modelfacts_system',
   'wos_modeldeployment_information',
   'modelfacts_user',
   'notebook_experiment'],
  'asset_id': 'c560f9d4-5ec

## Remove custom facts<a name="remove"></a> 

You can remove one or multiple facts by asscociated fact ids. This does not delete the attribute as a whole rather removes the associated value(s).

In [67]:
external_model.remove_custom_fact(fact_id="TrainingData_Size")

2022/12/12 17:19:23 INFO :  Value of Fact id TrainingData_Size removed successfully


In [68]:
external_model.remove_custom_facts(fact_ids=["TrainingData_Ratio","TestData_Size","TestData_Ratio","Train_Class_Distributions"])

2022/12/12 17:19:24 INFO : Values of Fact ids ['TrainingData_Ratio', 'TestData_Size', 'TestData_Ratio', 'Train_Class_Distributions'] removed successfully


In [69]:
ui_url = external_model.get_info(verbose=True)["url"]
display(Markdown("[Click here to see the external model and it's custom fact values after removing ids in the UI](" + ui_url + ")"))

[Click here to see the external model and it's custom fact values after removing ids in the UI](https://dataplatform.cloud.ibm.com/data/catalogs/745c21d7-ab6f-4408-9b3b-5d4760ccee1a/asset/c560f9d4-5ec7-431c-914b-394241f3f39a/asset-preview?context=cpdaas)

[back to top](#introduction)
## Add model to model usecase<a name="add_mu"></a> 

- Model use case helps to track model asset lifecycle across different environments like development,pre-production and production.
- You can link a model to existing model use case by providing `model_usecase_id` or create a new use case and link to that by providing `model_usecase_name` and `model_usecase_desc`.
- You can link to existing model use case by invoking `external_model.add_tracking_model_usecase(model_usecase_id="<model usecase id>")`
- If `model_usecase_catalog_id` is not given, it uses the default available platform asset catalog id. You need to have `EDITOR` access to platform asset catalog.
- When you open the model use case in model inventory, `model_usecase_id` refers to value after `asset/{ usecase id}` in browser URL.
- You can also invoke `get_tracking_model_usecase()` to return currently tracked model use case and get model use case details by calling `get_tracking_model_usecase().get_info()`
- If using Cloud pak for data, make sure Openpages integration is disabled in your platform (create catalog permission needed). You can check it from UI `(Model inventory -> Manage)`.

In [70]:
# creating new model usecase and link to that
external_model.add_tracking_model_usecase(model_usecase_name="ModelUsecaseDraft",model_usecase_desc="Draft model usecase for testing")

2022/12/12 17:19:24 INFO : Initiate linking model to new model use case......
2022/12/12 17:19:38 INFO : Successfully finished linking Model c560f9d4-5ec7-431c-914b-394241f3f39a to model use case


{'model_entry_catalog_id': '745c21d7-ab6f-4408-9b3b-5d4760ccee1a',
 'model_entry_id': '09e83de9-873e-4695-85e1-8411c0f0c3b1',
 'model_entry_name': 'ModelUsecaseDraft',
 'model_entry_status': 'draft'}

[back to top](#introduction)
## Create custom facts definitions for Model usecase facts type
<a name="customfactsdefmu"></a> 
- As you set facts definitions on model asset, you can also set it to model usecase level which is saved in catalog.
- You can change the header title (default is `Additional details`) by providing `section_name`.
- Facts type for model usecase is `model_entry_user`

In [71]:
!rm Asset_type_definition_me.csv
wget.download("https://raw.githubusercontent.com/IBM/ai-governance-factsheet-samples/main/Assets/data/Asset_type_definition_me.csv")

rm: Asset_type_definition_me.csv: No such file or directory


'Asset_type_definition_me.csv'

In [72]:
facts_client.assets.create_custom_facts_definitions(csvFilePath="Asset_type_definition_me.csv",section_name="Custom Model Details Title", type_name="model_entry_user")
time.sleep(5)

2022/12/12 17:19:39 INFO : Creating definitions for type model_entry_user
2022/12/12 17:19:40 INFO : Custom facts definitions updated Successfully


### Get custom facts definitions<a name="getcustomfactsdefmu"></a> 

In [73]:
facts_client.assets.get_facts_definitions(type_name="model_entry_user")

{'description': 'The model fact user asset type to capture user defined attributes.',
 'fields': [{'key': 'model_scope',
   'type': 'string',
   'facet': False,
   'is_array': False,
   'search_path': 'model_scope',
   'is_searchable_across_types': False},
  {'key': 'model_purpose',
   'type': 'string',
   'facet': False,
   'is_array': False,
   'search_path': 'model_purpose',
   'is_searchable_across_types': False},
  {'key': 'model_use',
   'type': 'string',
   'facet': False,
   'is_array': True,
   'search_path': 'model_use',
   'is_searchable_across_types': False},
  {'key': 'model_sponsor',
   'type': 'string',
   'facet': False,
   'is_array': False,
   'search_path': 'model_sponsor',
   'is_searchable_across_types': False}],
 'relationships': [],
 'properties': {'model_scope': {'type': 'string',
   'description': 'Model Scope',
   'placeholder': 'Model Scope',
   'is_array': False,
   'required': True,
   'hidden': False,
   'readonly': False,
   'default_value': '',
   'minim

[back to top](#introduction)
## Get model usecase<a name="get_mu"></a> 
- You can get to model usecase [helpers](https://ibm-aigov-facts-client.mybluemix.net/#model-use-case-helpers) in two ways.
    - Using `facts_client.assets.get_model_usecase()` utility returns a model usecase object. It requires model_usecase_id and catalaog id (Optional)
    <br>
    
    ```
    model_usecase=facts_client.assets.get_model_usecase(model_usecase_id=model_usecase.get_info()["model_usecase_id"])
    model_usecase=facts_client.assets.get_model_usecase(model_usecase_id=model_usecase.get_info()[model_usecase_id],catalog_id=model_usecase.get_info()["catalog_id"])
    ```
    
    <br>

    - Model usecase id and usecase catalog id can be fetched from UI. When you open the model usecase in model inventory, usecase id refers to value after `asset/{ usecase id}` in browser URL. Catalog id refers refers to value after `catalogs/{catalog id}` in browser URL.
    - If getting current model usecase, it returns a model usecase object allows to use all usecase [helpers](https://ibm-aigov-facts-client.mybluemix.net/#ibm_aigov_facts_client.factsheet.asset_utils_model.ModelAssetUtilities.get_tracking_model_usecase).


In [74]:
model_usecase=external_model.get_tracking_model_usecase()
model_usecase.get_info()

{'model_usecase_id': '09e83de9-873e-4695-85e1-8411c0f0c3b1',
 'container_type': 'catalog',
 'catalog_id': '745c21d7-ab6f-4408-9b3b-5d4760ccee1a',
 'facts_type': 'model_entry_user'}

[back to top](#introduction)
## Set custom facts for model usecase<a name="set_mu_custom"></a> 

- Custom facts related to model usecase like `Model usecase`, `Model Purpose` etc. can be set here.

In [75]:
model_usecase.set_custom_fact(fact_id="model_scope",value="Internal systems")
model_usecase.set_custom_fact(fact_id="model_purpose",value="Predicting risk factor when giving loans")
model_usecase.set_custom_fact(fact_id="model_sponsor",value="IBM")

2022/12/12 17:19:47 INFO : Custom fact model_scope successfully set to new value Internal systems
2022/12/12 17:19:47 INFO : Custom fact model_purpose successfully set to new value Predicting risk factor when giving loans
2022/12/12 17:19:48 INFO : Custom fact model_sponsor successfully set to new value IBM


In [76]:
model_usecase.set_custom_fact(fact_id="model_use",value=["Determining risk factors","Client debt collections"])

2022/12/12 17:19:48 INFO : Custom fact model_use successfully set to new value ['Determining risk factors', 'Client debt collections']


[back to top](#introduction)
## Get custom facts for model usecase<a name="get_mu_custom"></a> 

- You can get all facts or fact by given name used in facts definitions. You can also visit the UI to view the customs facts set. You can get URL by running `model_usecase.get_info(verbose=True)["url"]`

In [77]:
ui_url = model_usecase.get_info(verbose=True)["url"]
display(Markdown("[Click here to see the external model use case and it's custom facts values in the UI](" + ui_url + ")"))

[Click here to see the external model use case and it's custom facts values in the UI](https://dataplatform.cloud.ibm.com/data/catalogs/745c21d7-ab6f-4408-9b3b-5d4760ccee1a/asset/09e83de9-873e-4695-85e1-8411c0f0c3b1?context=cpdaas)

In [78]:
model_usecase.get_custom_fact_by_id(fact_id="model_scope")

'Internal systems'

In [79]:
model_usecase.get_custom_facts()

{'model_scope': 'Internal systems',
 'model_purpose': 'Predicting risk factor when giving loans',
 'model_sponsor': 'IBM',
 'model_use': ['Determining risk factors', 'Client debt collections']}

[back to top](#introduction)
## Get all facts<a name="get_all_mu"></a> 

In [80]:
model_usecase.get_all_facts()

{'metadata': {'usage': {'last_updated_at': '2022-12-12T22:19:48Z',
   'last_updater_id': 'IBMid-310002F0G1',
   'last_update_time': 1670883588679,
   'last_accessed_at': '2022-12-12T22:19:48Z',
   'last_access_time': 1670883588679,
   'last_accessor_id': 'IBMid-310002F0G1',
   'access_count': 0},
  'rov': {'mode': 0,
   'collaborator_ids': {},
   'member_roles': {'IBMid-310002F0G1': {'user_iam_id': 'IBMid-310002F0G1',
     'roles': ['OWNER']}}},
  'name': 'ModelUsecaseDraft',
  'description': 'Draft model usecase for testing',
  'asset_type': 'model_entry',
  'origin_country': 'us',
  'rating': 0.0,
  'total_ratings': 0,
  'catalog_id': '745c21d7-ab6f-4408-9b3b-5d4760ccee1a',
  'created': 1670883566006,
  'created_at': '2022-12-12T22:19:26Z',
  'owner_id': 'IBMid-310002F0G1',
  'size': 0,
  'version': 2.0,
  'asset_state': 'available',
  'asset_attributes': ['model_entry', 'modelfacts_global', 'model_entry_user'],
  'asset_id': '09e83de9-873e-4695-85e1-8411c0f0c3b1',
  'asset_category'

In [81]:
# Returns all model_entry_user facts 
model_usecase.get_facts_by_type()

{'model_scope': 'Internal systems',
 'model_purpose': 'Predicting risk factor when giving loans',
 'model_sponsor': 'IBM',
 'model_use': ['Determining risk factors', 'Client debt collections']}

[back to top](#introduction)
## Remove custom facts for model usecase<a name="remove_mu_custom"></a> 

In [82]:
model_usecase.remove_custom_fact(fact_id="model_scope")

2022/12/12 17:19:50 INFO :  Value of Fact id model_scope removed successfully


In [83]:
model_usecase.remove_custom_facts(fact_ids=["model_purpose","model_sponsor","model_use"])

2022/12/12 17:19:51 INFO : Values of Fact ids ['model_purpose', 'model_sponsor', 'model_use'] removed successfully


[back to top](#introduction)
## Get tracked models for model usecase<a name="get_tracked_mu"></a> 
- It returns all models tracked under current model usecase

In [84]:
model_usecase.get_tracked_models()

[{'id': 'c560f9d4-5ec7-431c-914b-394241f3f39a',
  'name': 'PytorchExternalModel',
  'type': 'external_model',
  'container_id': '745c21d7-ab6f-4408-9b3b-5d4760ccee1a',
  'container_name': 'Development space',
  'container_type': 'catalog',
  'is_deleted': False,
  'master_id': 'ddefabbde572cbfe759f54c61106718d',
  'source_asset_id': '',
  'algorithm': '',
  'deployment_space_type': 'development',
  'deployments': [{'id': 'fb469d7ef430b0baf0cab6c436e70375',
    'status': 'Pending Evaluation',
    'name': 'german_credit_risk_deployment',
    'model_revision_id': '',
    'model_revision_commit_message': '',
    'is_deleted': False,
    'has_alert': False,
    'alerts': []}]}]

[back to top](#introduction)
## Remove model from model usecase<a name="remove_mu"></a> 

In [85]:
external_model.remove_tracking_model_usecase()

2022/12/12 17:19:52 INFO : Successfully finished unregistering WKC Model c560f9d4-5ec7-431c-914b-394241f3f39a from model use case.


[back to top](#introduction)

## Cleanup<a name="clean"></a> 

In [86]:
# remove model usecase and model asset
if run_cleanup_at_end:
    facts_client.assets.remove_asset(asset_id=model_usecase.get_info()["model_usecase_id"],container_type=model_usecase.get_info()["container_type"],container_id=model_usecase.get_info()["catalog_id"])
    facts_client.assets.remove_asset(asset_id=external_model.get_info()["asset_id"],container_type=external_model.get_info()["container_type"],container_id=external_model.get_info()["container_id"])
else:
    model_ui_url = external_model.get_info(verbose=True)["url"]
    display(Markdown("[Click here to see the created external model details in UI](" + model_ui_url + ")"))
    model_usecase_ui_url = model_usecase.get_info(verbose=True)["url"]
    display(Markdown("[Click here to see the created external model use case in UI](" + model_usecase_ui_url + ")"))

[Click here to see the created external model details in UI](https://dataplatform.cloud.ibm.com/data/catalogs/745c21d7-ab6f-4408-9b3b-5d4760ccee1a/asset/c560f9d4-5ec7-431c-914b-394241f3f39a/asset-preview?context=cpdaas)

[Click here to see the created external model use case in UI](https://dataplatform.cloud.ibm.com/data/catalogs/745c21d7-ab6f-4408-9b3b-5d4760ccee1a/asset/09e83de9-873e-4695-85e1-8411c0f0c3b1?context=cpdaas)

[back to top](#introduction)

Congratulations. You have successfully finished the end to end sample around custom facts utilities like creating and getting facts definitions for model asset and model usecase, tracking/untracking a model to model usecase and set/get/remove helpers for custom facts for model and model usecase.

Copyright © 2020, 2022 IBM. This notebook and its source code are released under the terms of the MIT License.